In [1]:
%%time
import os
if not os.path.isdir('urban_oculus'):
  !git clone https://github.com/tgandor/urban_oculus
!(cd urban_oculus; git pull)
%run urban_oculus/snippets/get_coco_val2017_nounzip.ipynb
%run urban_oculus/snippets/validate_2017_functions.py

Already up to date.
CPU times: user 1.73 ms, sys: 511 µs, total: 2.24 ms
Wall time: 111 ms
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
CPU times: user 1.05 s, sys: 518 ms, total: 1.57 s
Wall time: 2.19 s


In [4]:
def validate_coco2017(model_config, quality, output_dir):
  result_file = result_path(model_config, quality, output_dir)
  if os.path.exists(result_file):
    print(time.strftime('%Y-%m-%d %H:%M:%S'), 'result seems to exist:', result_file)
    print('skipping', model_config, quality)
    return

  print(time.strftime('%Y-%m-%d %H:%M:%S'), 'unpacking')
  !unzip -o val2017.zip -d datasets/coco 2>&1 | tqdm --total 5000 >/dev/null
  # save_filesizes('filesizes_original.json', 'original', 0, output_dir)

  print(time.strftime('%Y-%m-%d %H:%M:%S'), f'degrading quality to {quality}')
  start = time.time()
  !mogrify -verbose -quality {quality} datasets/coco/val2017/*.jpg 2>&1 | tqdm --total 10000 >/dev/null
  elapsed = time.time() - start
  # save_filesizes(f'filesizes_{quality}.json', quality, elapsed, output_dir)

  print(time.strftime('%Y-%m-%d %H:%M:%S'), 'finished', quality)
  results, inference_time =  run_validation(model_config)
  
  print(time.strftime('%Y-%m-%d %H:%M:%S'), 'finished', quality)
  save_results(model_config, quality, results, inference_time, output_dir)

In [5]:
torch.cuda.set_device(0)
torch.cuda.current_device()

0

In [6]:
MODEL_ZOO_CONFIGS

['COCO-Detection/faster_rcnn_R_50_C4_3x.yaml',
 'COCO-Detection/faster_rcnn_R_50_DC5_3x.yaml',
 'COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml',
 'COCO-Detection/faster_rcnn_R_101_C4_3x.yaml',
 'COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml',
 'COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml',
 'COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml',
 'COCO-Detection/retinanet_R_50_FPN_3x.yaml',
 'COCO-Detection/retinanet_R_101_FPN_3x.yaml']

In [ ]:
%%time
# https://github.com/conansherry/detectron2/blob/master/configs/COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml
output_dir = '.' # '/content/gdrive/My Drive/results'  # mounted or not, here I come
#model_idx = 5
#model_config = MODEL_ZOO_CONFIGS[model_idx]

for model_idx, model_config in enumerate(reversed(MODEL_ZOO_CONFIGS)):
    print('model config:', model_config, 'idx:', model_idx)
    for q in tqdm(range(1, 101)):
      validate_coco2017(model_config, q, output_dir)

model config: COCO-Detection/retinanet_R_101_FPN_3x.yaml idx: 0


2020-12-19 00:28:40 unpacking
5001it [00:05, 868.00it/s]                                                      
2020-12-19 00:28:46 degrading quality to 1
  0%|                                      | 1/10000 [00:00<00:00, 35848.75it/s]
2020-12-19 00:28:46 finished 1


Loading config /usr/local/lib/python3.8/dist-packages/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.

model_final_971ab9.pkl: 0.00B [00:00, ?B/s]
model_final_971ab9.pkl:   0%|          | 8.19k/228M [00:00<6:59:30, 9.07kB/s]
model_final_971ab9.pkl:   0%|          | 24.6k/228M [00:01<5:05:14, 12.5kB/s]
model_final_971ab9.pkl:   0%|          | 57.3k/228M [00:01<3:39:32, 17.3kB/s]
model_final_971ab9.pkl:   0%|          | 139k/228M [00:01<2:35:57, 24.4kB/s] 
model_final_971ab9.pkl:   0%|          | 303k/228M [00:01<1:50:19, 34.5kB/s]
model_final_971ab9.pkl:   0%|          | 614k/228M [00:01<1:17:40, 48.9kB/s]
model_final_971ab9.pkl:   1%|          | 1.25M/228M [00:01<54:31, 69.4kB/s] 
model_final_971ab9.pkl:   1%|          | 2.52M/228M [00:02<38:05, 98.8kB/s]
model_final_971ab9.pkl:   2%|▏         | 4.62M/228M [00:02<26:30, 141kB/s] 
model_final_971ab9.pkl:   3%|▎         | 6.54M/2

Loading and preparing results...
DONE (t=1.06s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
COCOeval_opt.evaluate() finished in 9.44 seconds.
Accumulating evaluation results...
COCOeval_opt.accumulate() finished in 1.26 seconds.
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.404
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.603
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.432
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.240
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.443
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.522
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.336
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.532
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.56

Loading config /usr/local/lib/python3.8/dist-packages/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


...

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.


Loading and preparing results...
DONE (t=1.13s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
COCOeval_opt.evaluate() finished in 9.26 seconds.
Accumulating evaluation results...
COCOeval_opt.accumulate() finished in 1.26 seconds.
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.387
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.580
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.415
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.233
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.423
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.503
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.326
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.522
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.55

Loading config /usr/local/lib/python3.8/dist-packages/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


2020-12-19 22:06:10 finished 22


COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.


In [2]:
import torch

print('Device count:', torch.cuda.device_count())
torch.cuda.set_device(1)

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name())
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3, 1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3, 1), 'GB')

!head -n 11 /proc/cpuinfo


Device count: 2
Using device: cuda

GeForce GTX 1060 6GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Core(TM) i9-7900X CPU @ 3.30GHz
stepping	: 4
microcode	: 0x2000065
cpu MHz		: 4013.715
cache size	: 14080 KB
physical id	: 0
siblings	: 20


In [10]:
!ls

18.py			      NewValidate_multiple_JPEG.ipynb
22.py			      nowe_id
annotations_trainval2017.zip  pycardio
AoC22.ipynb		      pytony
datasets		      Scans
environment_testing.ipynb     stare_id
euler_spoj		      swapfile
hemodynamic-pipe	      TrainSet
in18.txt		      urban_oculus
in22.txt		      val2017_faster_rcnn_R_101_FPN_3x_q_5.json
lakeit			      val2017.zip
log_18.txt		      Zarr_scratch.ipynb
meats
